In [23]:
import librosa
import glob
import random

from librosa import display

import scipy as sp
import numpy as np

import matplotlib.pyplot as plt

import IPython.display as ipd

import time

In [2]:
import threading
import queue

In [3]:
threading.Thread

threading.Thread

In [4]:
file_list = sorted(glob.glob('../resource/*.wav'))

In [5]:
file = random.choice(file_list)

print(file)

../resource/17. DDU-DU DDU-DU (O-B).wav


In [24]:
def file2stream(file, sec, offset=0):
    
    fs = librosa.get_samplerate(file)
    
    stream = librosa.core.stream(file, 1, sec * fs, sec * fs, mono=False, offset=offset, fill_value=0)
    
    return stream

class StreamFeeder():
    
    def __init__(self, file_list):
        self.file_list = file_list
        self.sec = 5
        self.queue = queue.Queue(maxsize=20)
        self.batch_size = 16
        self.end_of_batching = False
        
    def shuffle(self):
        random.shuffle(self.file_list)
        
    def start_stream_feeding(self):
        
        t = threading.Thread(target=self.stream_feeding, args=())
        
        t.start()
        
        return
        
    def stream_feeding(self):
        
        self.shuffle()
        self.end_of_batching = False
        
        input_specgram_list = list()
        output_specgram_list = list()
        
        for file in self.file_list:
            offset = 5 * random.random()
            for data in file2stream(file, self.sec, offset):
                T, F, Sxx = sp.signal.stft(data, fs=16000, nperseg=400)
                Zxx = np.abs(Sxx)
                spectrogram = 20 * np.log10(np.maximum(Zxx, 1e-8)) / 160
                
                input_specgram = spectrogram[0, :, :]
                output_specgram = spectrogram[1, :, :]
                
                input_specgram_list.append(input_specgram)
                output_specgram_list.append(output_specgram)
                
                if len(input_specgram_list) >= self.batch_size:
                    try:
                        self.queue.put((np.stack(input_specgram_list), np.stack(output_specgram_list)))
                    except:
                        for spec in input_specgram_list:
                            print(spec.shape)
                        
                        for spec in output_specgram_list:
                            print(spec.shape)
                    
                    input_specgram_list = list()
                    output_specgram_list = list()
        
        if len(input_specgram_list) > 0:
            self.queue.put((np.stack(input_specgram_list), np.stack(input_specgram_list)))
            input_specgram_list = list()
            output_specgram_list = list()
            
        self.end_of_batching = True
    
    def batch_generator(self):
        
        while not self.end_of_batching or not self.queue.empty():
            
            try:
                yield self.queue.get_nowait()
            except:
                time.sleep(1)
                
        return
            

In [25]:
stream_feeder = StreamFeeder(file_list)

In [33]:
stream_feeder.start_stream_feeding()

In [37]:
batch[0].shape

(16, 201, 401)

In [39]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ConvModel(nn.Module):
    
    def __init__(self):
        super(ConvModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, 3, )
        
    def forward(self, tensor):
    

In [35]:
for i, batch in enumerate(stream_feeder.batch_generator()):
    print(i)
    break

0


Exception ignored in: <generator object StreamFeeder.batch_generator at 0x1c2916fed0>
RuntimeError: generator ignored GeneratorExit


In [ ]:
sec = 5

offset=0

fs = librosa.get_samplerate(file)

print('Sampling Rate: {}'.format(fs))

stream = librosa.core.stream(file, 1, sec * fs, sec * fs, mono=False, offset=offset)

# next(stream)

In [ ]:
def spec2mel(img):
    return 20 * np.log(np.maximum(img, 1e-8))

In [ ]:
# data = next(stream)

# T, F, Sxx = sp.signal.stft(data, fs=16000, nperseg=400)

# Zxx = np.abs(Sxx)

# plt.figure()
# plt.imshow(spec2mel(Zxx[0]), origin='reversed', aspect='auto')
# plt.show()

# plt.figure()
# plt.imshow(spec2mel(Zxx[1]), origin='reversed', aspect='auto')
# plt.show()

# print(Sxx.shape)

In [ ]:
data = next(stream)

In [ ]:
ipd.Audio(data, rate=16000)

In [ ]:
T, F, Sxx = sp.signal.stft(data, fs=16000, nperseg=400)

Zxx_origin = np.abs(Sxx)

Zxx[0] = np.abs(librosa.stft(y1, n_fft=400, hop_length=200, win_length=400))

In [ ]:
plt.plot(Zxx.flatten() / len(y1))

In [ ]:
# data = next(stream)

y1 = np.copy(data[0, :])
y2 = np.copy(data[1, :])

T, F, Sxx = sp.signal.stft(data, fs=16000, nperseg=400)

Zxx = np.abs(Sxx)
Zxx_recon = np.zeros(Zxx.shape)

Zxx[0] = np.abs(librosa.stft(y1, n_fft=400, hop_length=200, win_length=400))
Zxx[1] = np.abs(librosa.stft(y2, n_fft=400, hop_length=200, win_length=400))

data_1 = librosa.core.griffinlim(Zxx[0], n_iter=64, win_length=400, hop_length=200)
data_2 = librosa.core.griffinlim(Zxx[1], n_iter=64, win_length=400, hop_length=200)

Zxx_recon[0] = np.abs(librosa.stft(data_1, n_fft=400, hop_length=200, win_length=400))
Zxx_recon[1] = np.abs(librosa.stft(data_2, n_fft=400, hop_length=200, win_length=400))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

axes = axes.flatten()

axes[0].imshow(Zxx[0], aspect='auto', origin='reversed')
axes[1].imshow(Zxx_recon[0], aspect='auto', origin='reversed')
im = axes[2].imshow(Zxx[0] - Zxx_recon[0], aspect='auto', origin='reversed')
im.set_clim([-2, 2])


axes[3].imshow(Zxx[1], aspect='auto', origin='reversed')
axes[4].imshow(Zxx_recon[1], aspect='auto', origin='reversed')
im = axes[5].imshow(Zxx[1] - Zxx_recon[1], aspect='auto', origin='reversed')
im.set_clim([-2, 2])

plt.show()

## Highpass Filtering might help improving audio quality

fig, axes = plt.subplots(2, 1, figsize=(12, 12))

axes = axes.flatten()

axes[0].plot(data_1)
axes[0].plot(y1)

axes[1].plot(data_2)
axes[1].plot(y2)

plt.show()

